## EDA

In [421]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression

In [422]:
import warnings
warnings.filterwarnings("ignore")

In [423]:
df = pd.read_csv('papermodeldata(in).csv')

In [424]:
len(df.columns) -5 

51

In [425]:
df['ttd'].isnull().any()
df['fold'].isnull().any()
df['time'].isnull().any()
df['y'].isnull().any()

np.False_

In [426]:
count_labels = df['y'].groupby(df['y']).count()
count_labels[1]/count_labels[0]*100

np.float64(0.6785127250348604)

In [427]:
df['VitalID'].groupby(df['VitalID']).count()

VitalID
1005     12
1007    138
1010    267
1011     53
1013    103
       ... 
8769     13
8770      6
8771      4
8772      8
8773      4
Name: VitalID, Length: 5957, dtype: int64

In [428]:
df_dead = df[df['y'] == 1]
df_alive = df[df['y'] == 0]

In [429]:
df_alive['VitalID'].groupby(df_alive['VitalID']).count()

VitalID
1005     12
1007    138
1010    265
1011     53
1013     98
       ... 
8769     13
8770      6
8771      4
8772      8
8773      4
Name: VitalID, Length: 5948, dtype: int64

In [430]:
df_dead['VitalID'].groupby(df_dead['VitalID']).count()

VitalID
1010    2
1013    5
1017    7
1028    3
1031    3
       ..
8364    2
8501    1
8510    2
8521    3
8549    5
Name: VitalID, Length: 219, dtype: int64

In [431]:
count_folds = df.groupby(['fold','y']).size()
first = None
second = None
count = 0
for i in count_folds:
    count+=1
    if not(first):
        first = i
    else:
        second = i

    if(count==2):
        count = 0
        print((second/first)*100)
        first = None 
        second = None
    

0.674415267613718
0.5912786400591279
0.5741482208959734
0.7453205725417126
0.5463627848891663
0.7854225573358466
0.8354916868577157
0.5309466019417476
0.6714517249363279
0.8686388592968943


In [432]:
df_dropped = df.drop(['VitalID', 'time','fold','ttd'], axis=1)
df_na_dropped = df_dropped.dropna()
df_dead = df_dropped[df_dropped['y'] == 1]
df_alive = df_dropped[df_dropped['y'] == 0]
df_dead_na_dropped = df_na_dropped[df_na_dropped['y'] == 1]
df_alive_na_dropped = df_na_dropped[df_na_dropped['y'] == 0]

In [433]:
print(((len(df_dead)- len(df_dead_na_dropped)) /len(df_dead)) * 100)
print(((len(df_alive) - len(df_alive_na_dropped)) /len(df_alive))*100)

1.6073478760045925
2.0324221579976474


## Data Preparation

In [434]:
# keep the y seprate 
# remove nan values 
# remove duplicate values

In [435]:
# if ttd is there , that means the baby is eventually going to die..
# if ttd<=days , put y = 1
# if ttd>days  , put y =0
def format_y(no_of_days,ttd,y_values):
    for index in range(len(ttd)):
        if(ttd[index]<=no_of_days):
            y_values[index] = 1
        else:
            y_values[index] = 0
    return y_values

In [436]:
from scipy.stats.mstats import winsorize
def winsorize_col(column):
    return winsorize(column, limits=(0.001, .001))

def winsorize_df(df):
    return df.apply(winsorize_col,axis=0)

In [437]:
def create_df(df,days):
    ttd = df['ttd']
    y_arr = df['y']
    y_arr = format_y(days,ttd,y_arr)
    df['y'] = y_arr
    # change the y column in the df to the y_arr

    # we only care if the trainign data colun values contain NaN values 
    df_drop_col = df.drop(['VitalID','ttd','time'], axis=1)
    df_drop_col_dup = df_drop_col.drop_duplicates()
    df_drop_col_dup_na = df_drop_col_dup.dropna()

    y_arr = df_drop_col_dup_na['y']
    fold_arr = df_drop_col_dup_na['fold']

    df_drop_col_dup_na.drop(['y','fold'], axis=1,inplace=True)
    df = winsorize_df(df_drop_col_dup_na)
    return df,fold_arr,y_arr

In [438]:
df,fold_arr,y_arr = create_df(df,7)

In [439]:
Demographics = ['bwt', 'ega', 'male', 'apgar5']

HR = ['HR.SB.MotifTwo.diff.uu', 'HR.SB.MotifTwo.diff.uuu','HR.ST.LocalExtrema.l25.diffmaxabsmin', 'HR.std','HR.SB.MotifThree.diffquant.hhhh',
       'HR.EX.MovingThreshold.a0.25.b0.1.meanqover', 'HR.DN.cv.3','HR.SB.MotifTwo.iqr.ddd','HR.SY.StdNthDer.5','HR.ST.LocalExtrema.l25.maxmaxmed', 'HR.SY.StdNthDer.17', 'HR.Quantile.99', 'HR.FC.Suprise.tstat',
       'HR.SB.MotifThree.quantile.hhhh','HR.EX.MovingThreshold.a0.25.b0.05.meanqover','HR.PH.Walkerrunningvar...sw.meanabsdiff', 'HR.CO.tc3.1..denom', 'HR.mean',
       'HR.skew2','HR.SB.TransitionMatrix23.sumdiagcov','HR.ST.LocalExtrema.n100.minabsmin', 'HR.kurt2', 'HR.MF.arfit.sbc.7'
       ]

SPO2 = [ 'SP.EX.MovingThreshold.a0.25.b0.1.meanqover', 'SP.EX.MovingThreshold.a1.b0.25.iqrq', 'SP.PH.Walkerbiasprop.0.1..0.5..sw.meanabsdiff',
'SP.EX.MovingThreshold.a0.25.b0.1.maxq', 'SP.PH.Walkerprop.0.9..w.std', 'SP.skew2','SP.SB.MotifTwo.diff.dduu','SP.DN.RemovePointsmin.0.2.mean',
'SP.PH.Walkermomentum.2..sw.stdrat','SP.CO.tc3.1..denom','SP.mean','SP.kurt2', 'SP.SB.MotifThree.diffquant.hhh','SP.SB.MotifTwo.mean.dddd',
'SP.AutoCorr.lag.4','SP.SB.TransitionMatrix41.ondiag', 'SP.SB.MotifThree.quantile.baaa','SP.std', 'SP.SB.TransitionMatrix21.T10', 'SP.SB.BinaryMethod.iqr.pstretch1'
,'SP.PH.Walkerprop.0.9..sw.stdrat','SP.MF.arfit.sbc.7','SP.SB.TransitionMatrix22.mineig','SP.ST.LocalExtrema.n100.minabsmin']

In [440]:
def create_diff_dfs(df):
    df_hr = df[HR]
    df_spo2 = df[SPO2]
    df_demographics = df[Demographics]
    df_hr_spo2 = df[HR + SPO2] 
    df_hr_spo2_demographics = df[HR + SPO2 + Demographics]
    return df_hr,df_spo2,df_demographics,df_hr_spo2,df_hr_spo2_demographics

In [441]:
# while (curr_no_of_features(variable) >= 5)
    # for the different no of folds(variable)
        # take only that fold dataset from the df(variable) and its corresponding y for validation 
        # the rest of the data will be used to train the model

        # fit the model 
        # do the inference on the hold out set 
        # store the cv accuracy score 

        # call the permuation_importance function and provide the fitted model and hold out set as arguments 
        # store the feature importance of all the features 

    # average out the feature importance across the folds and get the argmin 
    # store the no of features 
    # store the average of the cv scores
    # store the standard deviation of the cv scores

    # no_of_features to decrease = min(5,curr_no_features-5)
    # if(no_of_features to decrease):
    #      break
    # remove the features from the dataset and continue with the changed dataset 

# Arguments: df(the dataset features), fold_arr, y_arr

In [ ]:
def give_results(df,fold_arr: list,y_arr: list):
    results = []
    folds = sorted(fold_arr.unique())
    curr_features = list(df.columns)

    while len(curr_features) >= 5:
        cv_scores = []
        feature_importances = []

        print("Current features: " + str(len(curr_features)))

        for fold_val in folds:

            # Split data
            X_val = df[fold_arr == fold_val][curr_features].to_numpy()
            y_val = y_arr[fold_arr == fold_val].to_numpy()

            X_train = df[fold_arr != fold_val][curr_features].to_numpy()
            y_train = y_arr[fold_arr != fold_val].to_numpy()

            # Fit model
            model = LogisticRegression(max_iter=1000).fit(X_train, y_train)
            y_pred = model.predict(X_val)
            acc = accuracy_score(y_val, y_pred)
            cv_scores.append(acc)

            # Permutation importance
            perm = permutation_importance(model, X_val, y_val, n_repeats=10, random_state=42)
            feature_importances.append(perm.importances_mean)

        # Average feature importance across folds
        avg_importance = np.mean(feature_importances, axis=0)
        # print(avg_importance)

        # Find least important features
        num_to_remove = min(5, len(curr_features) - 5)
        if num_to_remove == 0:
            break

        remove_idx = np.argsort(avg_importance)[:num_to_remove]
        remove_features = [curr_features[i] for i in remove_idx]

        # Store results
        results.append({
            'num_features': len(curr_features),
            'cv_score_mean': np.mean(cv_scores),
            'cv_score_std': np.std(cv_scores),
        })

        # Remove features for next iteration
        curr_features = [f for f in curr_features if f not in remove_features]

    # Convert results to DataFrame for inspection
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
df_results = give_results(df,fold_arr,y_arr)

Current features: 51
{'importances_mean': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.92847207e-05,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  7.32118017e-06,
        0.00000000e+00,  2.56241306e-04,  0.00000000e+00,  8.56578080e-04,
        0.00000000e+00, -6.58906216e-05,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  3.66059009e-04,
        0.00000000e+00,  0.00000000e+00, -7.32118017e-06,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.03528809e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  4.39270810e-05,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.46423603e-05,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])